In [1]:
from pathlib import Path
from tqdm import tqdm
import polars as pl

In [2]:
CACHE_DIR = 'cache'
USE_CHAT_HISTORY = False
PRINT_QA = False
TUTORIALS = [1, 2, 3, 4]
MODEL_NAME = "dragonfly"

In [3]:
if MODEL_NAME == "dragonfly":
    from models.dragonfly import Dragonfly
    model = Dragonfly(use_history=USE_CHAT_HISTORY, cache_dir=CACHE_DIR)
elif MODEL_NAME == "cogagent":
    from models.cogagent import CogAgent
    model = CogAgent(use_history=USE_CHAT_HISTORY, cache_dir=CACHE_DIR)
else:
    from models.internvl import InternVL
    assert MODEL_NAME in InternVL.VARIANTS.keys(), f"Keys are {InternVL.VARIANTS.keys()}"
    model = InternVL(model_name=MODEL_NAME, use_history=USE_CHAT_HISTORY, cache_dir=CACHE_DIR)

/home/drege/mambaforge/envs/dragonfly_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/drege/mambaforge/envs/dragonfly_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Initialize Vision Encoder


In [4]:
final_foldername = 'history' if USE_CHAT_HISTORY else 'no_history'
res_folder = Path('results') / MODEL_NAME.lower() / final_foldername
if not res_folder.is_dir():
    res_folder.mkdir(parents=True)

initial_prompt = "You are a gamer. You are playing a game tutorial. I will provide you some screenshots of the tutorial. Answer the questions related to the screenshot. Be concise and direct."

In [5]:
def get_img_path(basepath: Path, frame: str) -> str:
    filepath = basepath / f'{frame}.png'
    if not filepath.is_file():
        filepath = basepath / f'{frame}:.png'
    assert filepath.is_file(), f'File {str(filepath)} not found!'

    return filepath

In [6]:
for revision in ['8832ec1f3d1e27aebefa9228dbbc57474edd94cb', 'last']: 
    postfix_export = '' if not USE_CHAT_HISTORY else '_with_history'
    filename = f'{MODEL_NAME.lower()}_{revision[:7]}{postfix_export}'
    BASEPATH = Path(f'data/frames/{revision}')

    original = pl.read_csv(BASEPATH / "frame_labels.csv").drop_nulls()
    original = (
        original.group_by("frame")
        .agg(pl.col("*"))
        .with_columns(
            pl.exclude("frame")
            .map_elements(
                lambda x: [f"{n}) {q}" for q, n in zip(x, range(1, len(x) + 1))],
                return_dtype=pl.List(pl.Utf8),
            )
            .list.join("\n")
        )
    )

    responses = []
    for tutorial in TUTORIALS:
        model.clean_history()
        filtered = original.filter(pl.col("frame").str.starts_with(str(tutorial))).sort(
            "frame"
        )
        for i, (frame_label, questions, expectation) in tqdm(
            enumerate(filtered.iter_rows()), total=filtered.height
        ):
            img_path = get_img_path(BASEPATH, frame_label)

            question = (
                f"{initial_prompt}\n{questions}"
                if i == 0 or not USE_CHAT_HISTORY
                else questions
            )
            resp = model.generate(img_path, question)

            if PRINT_QA:
                print("@" * 10)
                print(f"Frame: {frame_label}")
                print(f"Question: {questions}")
                print(f"Expectation: {expectation}")
                print(f"Response: {resp}")
                print("@" * 10)

            resp = {
                "frame": frame_label,
                "question": questions,
                "expectation": expectation,
                "reply": resp,
            }

            responses.append(resp)
            if not USE_CHAT_HISTORY:
                model.clean_history()

    pl.DataFrame(responses).write_csv(res_folder / f"{filename}.csv")

  0%|          | 0/9 [00:00<?, ?it/s]/home/drege/mambaforge/envs/dragonfly_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 83%|████████▎ | 5/6 [00:09<00:02,  2.40s/it]

  0%|          | 0/10 [00:00<?, ?it/s]/home/drege/mambaforge/envs/dragonfly_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 11/11 [00:27<00:00,  2.53s/it]
